# Fellegi-Sunter Model for Entity Resolution

This notebook showcases how to go about doing entity resolution using probabilistic
methods. While the current notebook showcases a simple record linkage scenario,
there's no reason why the approach wouldn't work on any type of entity matching.
The main constraint is the existence of two data sources.

But let's not waste breath. Let's make a few imports.

In [80]:
import os
from itertools import product
from typing import Generator

import numpy as np
import polars as pl

from matchescu.matching.entity_reference import EntityReferenceComparisonConfig, FellegiSunterComparison
from matchescu.matching.ml.datasets import CsvDataSource, Traits

Next, we need to define some fairly important 'constants' (alas, there's no such thing in Python).

In [81]:
DATADIR = os.path.abspath("../../data")
LEFT_CSV_PATH = os.path.join(DATADIR, "abt-buy", "Abt.csv")
RIGHT_CSV_PATH = os.path.join(DATADIR, "abt-buy", "Buy.csv")
GROUND_TRUTH_PATH = os.path.join(DATADIR, "abt-buy", "abt_buy_perfectMapping.csv")

The sources of information can be structured in any way. However, when we read
from the data source we expect to be able to refer to discrete pieces of
information.
The important bit is to have a decent feature extraction process that is able to
produce relatively uniformly shaped entity references. That's what `Traits()`
do. That way we can get a neat matching process going.  

In [82]:
abt_traits = list(Traits().int([0]).string([1, 2]).currency([3]))
abt = CsvDataSource(name="abt", traits=abt_traits).read_csv(LEFT_CSV_PATH)
buy_traits = list(Traits().int([0]).string([1,2,3]).currency([4]))
buy = CsvDataSource(name="buy", traits=buy_traits).read_csv(RIGHT_CSV_PATH)
gt = set(pl.read_csv(os.path.join(DATADIR, "abt-buy", "abt_buy_perfectMapping.csv"), ignore_errors=True).iter_rows())

We've used a CSV data source, but you can easily see how you'd be able to use
anything as a data source, right? All you need to implement is a way to chunk
data from the source into discrete pieces which can be addressed either by
index or by name (let's call them 'records').
As it's been defined until now, entity resolution works only on discrete pieces
of information (so no streams, sorry) so we should be fine.
A model supporting streams would be great, tbh.
And if you're suddenly thinking about free text and how named entity recognition
and disambiguation might not fit into this picture, just think about how the
word has a 'value' (the word itself) and various attributes about the context
it was found in. So even free text is also, ultimately, devolved into discrete
'records'.

The second thing to notice is how the ground truth is just this set of true
matches. Meaning: pairs of identifiers of things from the data source we decide
refer to the same real-world entity are collected in this `set`.

After we've loaded the data sources and the ground truth, we can set up how
we're going to compare entity references. Comparing entity references can be
done in a myriad of ways. Generally speaking, though, it's always about
comparing attributes.
Yes, even when we're comparing vectorial representations of the entity
reference obtained with a transformer or some other technique. We could simply
say the vectors are a single attribute that we care about. Or we could say we
have `N` integer attributes, where `N` is the vector size. Each option has its
perks and its drawbacks.

But let's not digress. Here's some interesting observations on comparing entity
reference attributes:

* the F-S model allows specifying various comparison outcomes depending on the
conditions present in the data. If data is missing you can return a special code
which is different from a match or a non-match. Technically, the sky's the
limit: you could even bake the similarity into the return value provided
similarity is returned on a discrete scale (not a value from a continuous
interval). So stick to integers or letters of the alphabet, project and
approximate the similarity and you'll be fine. 
* other models might return different results during the comparisons. So we need
room for expansion: an F-S comparison is not the same as a naive Bayes
comparison (that one only allows `-1` and `1` as the result of a value
comparison). So these comparisons work with very different result types. F-S is
technically limited to a finite subset of integers, whereas Naive Bayes allows
only two specific values. If we extend this to other comparison types, we can
easily see results in the form of: floating point numbers, graph edges or nodes
and even context vectors.

Very long-winded, I agree. Let's specify how we're comparing attributes now.

In [83]:
fs_config = FellegiSunterComparison().levenshtein(
    "name", 1, 1, threshold=0.8
).levenshtein(
    "description", 2, 2, threshold=0.8
).exact(
    "price", 3, 4
)

Mkay. Attributes are 0-indexed (because `tuple`s are 0-indexed and entity
references are essentially tuples). So we've set up:

* `name` comparison between the 2nd attributes of two tuples, respectively,
* `description` comparision between their 3rd respective attributes, and
* `price` comparison between the 4th item of the left member of the comparison
and the 5th item of the right member.

Finally, it's time to do something with this information. So we're going to
build a data set for record linkage. Record linkage typically wants to find
which records from the first source refer to the same entity as corresponding
records from the second source.

To determine that, we have to do a lot of grunt work. First, we should extract
entity references with the process we defined a while back.
Next, we need to compare every reference extracted from the left data source to
every reference extracted from the right data source.
Finally, the results of the comparisons go into a feature matrix while the
information whether the references point to the same entity goes into a `target`
vector.

In [84]:
from matchescu.matching.ml.datasets import RecordLinkageDataSet

ds = RecordLinkageDataSet(abt, buy, gt)
y = ds.target_vector
X = ds.compute_feature_matrix(fs_config)

Next up, we introduce a function that splits the comparison data that we've just
computed in two: train and test data.

In [85]:
import sklearn.model_selection as sklearn

def train_test_split(complete_feature_data: pl.DataFrame, target_values: np.ndarray, train_size: float = 0.6) -> tuple[pl.DataFrame, ...]:
    X_train, X_test, y_train, y_test = sklearn.train_test_split(complete_feature_data.to_numpy(), target_values, train_size=train_size)
    concat_dfs = []
    feature_schema = {config.label: pl.Object for config in fs_config.specs}
    feature_schema["match_pattern"] = pl.String
    for f, t in [(X_train, y_train), (X_test, y_test)]:
        f_df = pl.DataFrame(f, feature_schema)
        t_df = pl.DataFrame(t, {"target": pl.Boolean})
        concat_dfs.append(pl.concat([f_df, t_df], how="horizontal"))
    return tuple(concat_dfs)

Let's see what the split looks like.

In [86]:
training_df, test_df = train_test_split(X, y)
y_ratio = len(y[y==1]) / len(y)
y_train_ratio = len(training_df.filter(pl.col("target"))) / len(training_df)
y_test_ratio = len(test_df.filter(pl.col("target"))) / len(test_df)
print(y_ratio, y_train_ratio, y_test_ratio)

0.0009293050458637878 0.000896549484589938 0.0009784383530891756


Notice how the ratio of true matches with respect to the data size is preserved
after the split. That's pretty helpful because now we can assume with some
confidence that the findings we have on the training data can be extrapolated to
the test data and beyond.

Up until now - excepting the comparisons defined only for the Fellegi-Sunter
model - things have been pretty generic. We could, in fact, train neural nets,
logistic regression models or Naive Bayes models on the comparison data.

From here on, though, we're going to take a pattern-based approach to entity
resolution. This is suitable for decision trees or the Fellegi-Sunter model.
Let's see what it's about.

In [87]:
from numpy import log2

from matchescu.matching.attribute import TernaryResult


def _generate_fs_patterns(n_comparisons: int) -> Generator[list[TernaryResult], None, None]:
    yield from product(TernaryResult, repeat=n_comparisons)


def _logarithmic_weight(prob_tp: float, prob_fp: float) -> float:
    if prob_fp and prob_tp:
        return log2(prob_tp / prob_fp)
    if prob_tp and not prob_fp:
        return 1000000
    return -1000000


def _rows_matching_pattern(pattern: tuple, cmp: EntityReferenceComparisonConfig):
    match_pattern_expr = pl.lit(True)
    for pattern_value, attr_comparison_spec in zip(pattern, cmp.specs):
        match_pattern_expr = match_pattern_expr & (pl.col(attr_comparison_spec.label) == pattern_value)
    return match_pattern_expr


def create_pattern_weights(training_data: pl.DataFrame, cmp: EntityReferenceComparisonConfig) -> pl.DataFrame:
    data = []
    n_comparisons = len(cmp.specs)
    for pattern in _generate_fs_patterns(n_comparisons):
        pattern_string = "".join(map(str, pattern))
        pattern_df = training_data.filter(pl.col("match_pattern") == pattern_string)
        total_pattern_matches = len(pattern_df)
        true_pattern_matches = len(pattern_df.filter(pl.col("target")))
        false_pattern_matches = len(pattern_df.filter(pl.col("target").not_()))
        true_pattern_match_prob, false_pattern_match_prob = 0, 0
        if total_pattern_matches != 0:
            true_pattern_match_prob = true_pattern_matches / total_pattern_matches
            false_pattern_match_prob = false_pattern_matches / total_pattern_matches

        pattern_weight = _logarithmic_weight(true_pattern_match_prob, false_pattern_match_prob)
        data.append({
            "pattern": "".join(map(str, pattern)),
            "tp": true_pattern_matches,
            "fp": false_pattern_matches,
            "count": total_pattern_matches,
            "P(tp|pattern)": true_pattern_match_prob,
            "P(fp|pattern)": false_pattern_match_prob,
            "pattern_weight": pattern_weight
        })

    tp_df = pl.DataFrame(data)
    return tp_df.sort(by="pattern_weight", descending=True)

Whoa! That was a lot of code! But what it does is quite easy.
A pattern is just a combination of comparison results.
 
Let's say we have 3 comparisons that we make and 3 possible results for each
comparison. We want to evaluate what are the odds of successfully identifying
the same real-world entity for every possible combination of the results of the
three comparisons. For example, we want to know what are the odds of identifying
a real-world entity in case there's a match on name, a mismatch on description
and data missing on price. Spoiler alert: quite high.

In [88]:
pattern_weight_df = create_pattern_weights(training_df, fs_config)
print(pattern_weight_df)

shape: (27, 7)
┌─────────┬─────┬─────┬───────┬───────────────┬───────────────┬────────────────┐
│ pattern ┆ tp  ┆ fp  ┆ count ┆ P(tp|pattern) ┆ P(fp|pattern) ┆ pattern_weight │
│ ---     ┆ --- ┆ --- ┆ ---   ┆ ---           ┆ ---           ┆ ---            │
│ str     ┆ i64 ┆ i64 ┆ i64   ┆ f64           ┆ f64           ┆ f64            │
╞═════════╪═════╪═════╪═══════╪═══════════════╪═══════════════╪════════════════╡
│ 101     ┆ 1   ┆ 0   ┆ 1     ┆ 1.0           ┆ 0.0           ┆ 1e6            │
│ 102     ┆ 9   ┆ 5   ┆ 14    ┆ 0.642857      ┆ 0.357143      ┆ 0.847997       │
│ 100     ┆ 23  ┆ 18  ┆ 41    ┆ 0.560976      ┆ 0.439024      ┆ 0.353637       │
│ 001     ┆ 7   ┆ 186 ┆ 193   ┆ 0.036269      ┆ 0.963731      ┆ -4.731804      │
│ 012     ┆ 1   ┆ 967 ┆ 968   ┆ 0.001033      ┆ 0.998967      ┆ -9.917372      │
│ …       ┆ …   ┆ …   ┆ …     ┆ …             ┆ …             ┆ …              │
│ 211     ┆ 0   ┆ 0   ┆ 0     ┆ 0.0           ┆ 0.0           ┆ -1e6           │
│ 212     ┆ 0

You'll notice that we've had quite some success with some of our patterns.
Alas, this is not the good news we're hoping it is. The patterns match very few
entities. So even if we're precise, we'll have very bad recall.
Let's see some stats before we jump to conclusions.

In [89]:
total_count = len(training_df)
entity_count = len(training_df.filter(pl.col("target")))
mismatch_count = total_count - entity_count
print(f"total={total_count}; entities={entity_count}; non-entities={mismatch_count}")
print(pattern_weight_df.select("pattern", "P(tp|pattern)", "P(fp|pattern)", "pattern_weight"))

total=708271; entities=635; non-entities=707636
shape: (27, 4)
┌─────────┬───────────────┬───────────────┬────────────────┐
│ pattern ┆ P(tp|pattern) ┆ P(fp|pattern) ┆ pattern_weight │
│ ---     ┆ ---           ┆ ---           ┆ ---            │
│ str     ┆ f64           ┆ f64           ┆ f64            │
╞═════════╪═══════════════╪═══════════════╪════════════════╡
│ 101     ┆ 1.0           ┆ 0.0           ┆ 1e6            │
│ 102     ┆ 0.642857      ┆ 0.357143      ┆ 0.847997       │
│ 100     ┆ 0.560976      ┆ 0.439024      ┆ 0.353637       │
│ 001     ┆ 0.036269      ┆ 0.963731      ┆ -4.731804      │
│ 012     ┆ 0.001033      ┆ 0.998967      ┆ -9.917372      │
│ …       ┆ …             ┆ …             ┆ …              │
│ 211     ┆ 0.0           ┆ 0.0           ┆ -1e6           │
│ 212     ┆ 0.0           ┆ 0.0           ┆ -1e6           │
│ 220     ┆ 0.0           ┆ 0.0           ┆ -1e6           │
│ 221     ┆ 0.0           ┆ 0.0           ┆ -1e6           │
│ 222     ┆ 0.0       

In [90]:
import plotly.graph_objs as go

fig = go.Figure(data=[
    go.Bar(x=pattern_weight_df["pattern"], y=pattern_weight_df["P(tp|pattern)"], name="true positives"),
    go.Bar(x=pattern_weight_df["pattern"], y=pattern_weight_df["P(fp|pattern)"], name="false positives"),
])
fig.update_layout(barmode="group")
fig.show()

Ok, that doesn't look very good. Indeed, the plot looks rather nice, but the
recall is very low according to the table.
What's worse, the patterns with good recall have dismal precision.
Of course, we can tune the comparisons we defined a while back at the very
beginning of our little foray, but that's tedious.

For completeness let's just perform the standard F-S decision process to see
some results and understand how truly badly we're doing if we're not putting
in significant amounts of work in tweaking attribute-level comparisons.

In [91]:
T_mu = 0  # the pattern represents a true link above this value 
T_lambda = -4  # the pattern represents a true non-link below this value

positive_patterns = set(pattern_weight_df.filter(pl.col("pattern_weight") >= T_mu)["pattern"].to_list())
negative_patterns = set(pattern_weight_df.filter(pl.col("pattern_weight") <= T_lambda)["pattern"].to_list())
undecided = set(pattern_weight_df.filter(pl.col("pattern_weight").is_between(T_lambda, T_mu))["pattern"].to_list())

print(positive_patterns, negative_patterns, undecided)

{'100', '102', '101'} {'220', '200', '002', '111', '112', '110', '000', '210', '122', '001', '202', '020', '010', '022', '121', '222', '021', '212', '012', '221', '011', '211', '120', '201'} set()


The decision process involves two thresholds: $T_\mu$ and $T_\lambda$.
The first threshold is the threshold for positive matching, while the second
is the threshold of negative non-matching. Anything above the first and below
the second yields a definitive answer in the F-S model. Anything between them
yields ambiguous results that need manual inspection and, for that reason,
won't be included in the calculation of precision and recall.

The two values are arbitrarily chosen. You can play around with them. The table
and plot we've created before should serve as a guide for choosing these values. 

In [92]:
print(test_df)

shape: (472_181, 5)
┌────────┬─────────────┬────────┬───────────────┬────────┐
│ name   ┆ description ┆ price  ┆ match_pattern ┆ target │
│ ---    ┆ ---         ┆ ---    ┆ ---           ┆ ---    │
│ object ┆ object      ┆ object ┆ str           ┆ bool   │
╞════════╪═════════════╪════════╪═══════════════╪════════╡
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
│ 0      ┆ 0           ┆ 2      ┆ 002           ┆ false  │
│ …      ┆ …           ┆ …      ┆ …             ┆ …      │
│ 0      ┆ 0           ┆ 2      ┆ 002           ┆ false  │
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
│ 0      ┆ 0           ┆ 2      ┆ 002           ┆ false  │
│ 0      ┆ 0           ┆ 2      ┆ 002           ┆ false  │
│ 0      ┆ 0           ┆ 0      ┆ 000           ┆ false  │
└────────┴─────────────┴────────┴───

In [93]:
total = len(test_df)

links_df = test_df.with_columns(
    [
        pl.col("match_pattern").is_in(positive_patterns).alias("is_link"),
        pl.col("match_pattern").is_in(negative_patterns).alias("non_link"),
        pl.col("match_pattern").is_in(undecided).alias("ambiguous"),
    ]
)
tp, fp, tn, fn, ambiguous = links_df.lazy().with_columns(
    tp=pl.col("is_link").and_(pl.col("target")),
    fp=pl.col("is_link").and_(pl.col("target").not_()),
    tn=pl.col("non_link").and_(pl.col("target").not_()),
    fn=pl.col("non_link").and_(pl.col("target")),
).select(
    "tp", "fp", "tn", "fn", "ambiguous"
).sum().collect().row(0)

In [94]:
unambiguous = total - ambiguous
print(
    f"total comparisons: {total}; manual inspections: {ambiguous}; unambiguous comparisons: {unambiguous}")
print(f"tp={tp};fp={fp};tn={tn};fn={fn}")
print(f"tp+fp+fn+tn={tp + fp + fn + tn}; unambiguous={unambiguous}")
print(f"manual inspections required: {ambiguous}")

total comparisons: 472181; manual inspections: 0; unambiguous comparisons: 472181
tp=29;fp=11;tn=471708;fn=433
tp+fp+fn+tn=472181; unambiguous=472181
manual inspections required: 0


In [95]:
p = tp / (tp + fp) if tp + fp > 0 else 0
r = tp / (tp + fn) if tp + fn > 0 else 0
f1 = 2 * p * r / (p + r) if p + r > 0 else 0
print(p, r, f1)

0.725 0.06277056277056277 0.11553784860557768


Not a very optimistic score! 
There's two options:

1. Play around with attribute level comparisons (implement fuzzy matching on price, for example)
2. Become better at approximating which comparisons yield true matches

To keep this in the realm of probabilities, let's try the Naive Bayes approach
next!